In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from huggingface_hub import login, HfApi
import warnings
import logging
import os

# Suppress warnings
warnings.filterwarnings('ignore')
logging.getLogger("transformers").setLevel(logging.ERROR)
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = '1'

print("✓ All imports successful")

✓ All imports successful


In [11]:
# ============================================================================
# HUGGING FACE CONFIGURATION
# ============================================================================

class HF_CONFIG:
    """Hugging Face Hub configuration"""
    
    # Your Hugging Face username and model repository name
    HUB_USERNAME = "AKG2"  # ← Change this to your HF username
    HUB_MODEL_NAME = "clinical-longformer-readmission-frozen"  # ← Different name for frozen model
    
    # This will be constructed as: username/model-name
    @property
    def HUB_MODEL_ID(self):
        return f"{self.HUB_USERNAME}/{self.HUB_MODEL_NAME}"
    
    # Kaggle secret name for HF token (don't change unless you named it differently)
    KAGGLE_SECRET_NAME = "HF_TOKEN"
    
    # Whether to push to hub during training
    PUSH_TO_HUB = True
    
    # Whether to make the repository private
    PRIVATE_REPO = False

# ============================================================================
# TRAINING CONFIGURATION
# ============================================================================

class CONFIG:
    """Training configuration"""
    
    # Data paths
    DATA_PATH = "/kaggle/input/mimic-iv"
    MODEL_NAME = "yikuan8/Clinical-Longformer"
    
    # Hugging Face configuration
    HF = HF_CONFIG()
    
    # Dataset settings
    SAMPLE_SIZE = 30000  # Set to None to use full dataset
    TEST_SIZE = 0.2
    RANDOM_SEED = 42
    
    # Model settings
    MAX_LENGTH = 4096
    FREEZE_BACKBONE = True  # ← NEW: Freeze backbone weights
    
    # Training hyperparameters (adjusted for frozen backbone)
    EPOCHS = 10  # Can train longer since only classifier is updating
    TRAIN_BATCH_SIZE = 4  # Can use larger batch size
    VALID_BATCH_SIZE = 8
    GRADIENT_ACCUMULATION = 8  # Reduced since we have larger batch size
    LEARNING_RATE = 1e-3  # Higher LR for classifier-only training
    WEIGHT_DECAY = 0.01
    WARMUP_RATIO = 0.1
    
    # GPU optimization
    USE_FP16 = True
    
    # Output directory
    OUTPUT_DIR = "./frozen_readmission_model"

In [12]:
# ============================================================================
# HUGGING FACE HUB LOGIN
# ============================================================================

def login_to_huggingface(config):
    """Log in to Hugging Face Hub using Kaggle secret"""
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        hf_token = user_secrets.get_secret(config.HF.KAGGLE_SECRET_NAME)
        
        if hf_token:
            login(token=hf_token, add_to_git_credential=True)
            print(f"✓ Successfully logged in to Hugging Face Hub")
            print(f"  Model will be pushed to: {config.HF.HUB_MODEL_ID}")
            return True
        else:
            print("⚠ HF_TOKEN not found in Kaggle secrets")
            return False
    except Exception as e:
        print(f"⚠ Could not log in to Hugging Face Hub: {e}")
        print("  Training will continue without Hub integration")
        return False


# ============================================================================
# DATA LOADING AND PREPARATION
# ============================================================================

def load_and_prepare_data(config):
    """Load and merge all data sources for readmission prediction"""
    print("=" * 80)
    print("LOADING DATA")
    print("=" * 80)
    
    # Load main admissions file
    admissions = pd.read_csv(f"{config.DATA_PATH}/admissions_with_readmission_labels.csv")
    print(f"✓ Loaded admissions: {admissions.shape}")
    
    # Load discharge notes
    discharge = pd.read_csv(f"{config.DATA_PATH}/discharge_notes-001.csv")
    print(f"✓ Loaded discharge notes: {discharge.shape}")
    
    # Load radiology notes
    radiology = pd.read_csv(f"{config.DATA_PATH}/radiology_notes.csv")
    print(f"✓ Loaded radiology notes: {radiology.shape}")
    
    # Combine notes: concatenate discharge and radiology by hadm_id
    print("\nCombining notes...")
    discharge_grouped = discharge.groupby('hadm_id')['text'].apply(
        lambda x: ' '.join(x.astype(str))
    ).reset_index()
    discharge_grouped.columns = ['hadm_id', 'discharge_text']
    
    radiology_grouped = radiology.groupby('hadm_id')['text'].apply(
        lambda x: ' '.join(x.astype(str))
    ).reset_index()
    radiology_grouped.columns = ['hadm_id', 'radiology_text']
    
    # Merge notes
    notes_combined = discharge_grouped.merge(
        radiology_grouped, on='hadm_id', how='outer'
    )
    
    # Combine all text
    notes_combined['combined_text'] = (
        notes_combined['discharge_text'].fillna('') + ' ' + 
        notes_combined['radiology_text'].fillna('')
    )
    notes_combined['combined_text'] = notes_combined['combined_text'].str.strip()
    
    # Merge with admissions
    df = admissions.merge(notes_combined[['hadm_id', 'combined_text']], 
                          on='hadm_id', how='left')
    
    # Use combined_text if available, otherwise use original text
    df['final_text'] = df['combined_text'].fillna(df.get('text', ''))
    df['final_text'] = df['final_text'].fillna('')
    
    print(f"✓ Final merged data: {df.shape}")
    
    # Sample data if needed
    if config.SAMPLE_SIZE is not None:
        df = df.sample(n=min(config.SAMPLE_SIZE, len(df)), 
                       random_state=config.RANDOM_SEED)
        print(f"✓ Sampled {len(df)} rows for testing")
    
    # Prepare readmission labels (binary: 0 or 1)
    print("\nPreparing labels...")
    df['readmitted_30day'] = df['readmitted_30day'].astype(int)
    
    print(f"  - Readmission distribution: {df['readmitted_30day'].value_counts().to_dict()}")
    
    # Calculate class weights for readmission (handle imbalance)
    pos_count = df['readmitted_30day'].sum()
    neg_count = len(df) - pos_count
    pos_weight = neg_count / pos_count if pos_count > 0 else 1.0
    
    print(f"  - Positive weight for readmission: {pos_weight:.2f}")
    
    return df, pos_weight


# ============================================================================
# DATASET CLASS
# ============================================================================

class ReadmissionDataset(Dataset):
    """Dataset for single-task readmission prediction"""
    
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        # Tokenize
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }


# ============================================================================
# FROZEN BACKBONE MODEL
# ============================================================================

class FrozenReadmissionModel(nn.Module):
    """Model with frozen backbone - only classifier is trainable"""
    
    def __init__(self, model_name, freeze_backbone=True):
        super().__init__()
        
        # Backbone: Clinical-Longformer
        self.backbone = AutoModel.from_pretrained(model_name)
        hidden_size = self.backbone.config.hidden_size
        
        # Freeze backbone if specified
        if freeze_backbone:
            print("\n🔒 FREEZING BACKBONE WEIGHTS")
            for param in self.backbone.parameters():
                param.requires_grad = False
            
            # Count frozen parameters
            frozen_params = sum(p.numel() for p in self.backbone.parameters())
            print(f"  - Frozen parameters: {frozen_params:,}")
        
        # Single classification head for readmission (TRAINABLE)
        self.classifier = nn.Linear(hidden_size, 1)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.1)
        
        # Count trainable parameters
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in self.parameters())
        print(f"  - Trainable parameters: {trainable_params:,}")
        print(f"  - Total parameters: {total_params:,}")
        print(f"  - Trainable %: {100 * trainable_params / total_params:.2f}%")
    
    def forward(self, input_ids, attention_mask, labels=None, **kwargs):
        # Get [CLS] token representation from backbone (no gradient)
        with torch.set_grad_enabled(self.training):
            outputs = self.backbone(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
        
        # Use [CLS] token (first token)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        
        # Readmission prediction (this layer is trainable)
        logits = self.classifier(cls_output)
        
        return {'logits': logits}


# ============================================================================
# CUSTOM TRAINER WITH WEIGHTED LOSS
# ============================================================================

class WeightedLossTrainer(Trainer):
    """Custom trainer with weighted BCE loss for class imbalance"""
    
    def __init__(self, pos_weight, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = torch.tensor([pos_weight])
        self.bce_loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Move pos_weight to correct device
        model_device = next(model.parameters()).device
        if self.pos_weight.device != model_device:
            self.pos_weight = self.pos_weight.to(model_device)
            self.bce_loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)
        
        # Extract labels
        labels = inputs.pop('labels')
        
        # Forward pass
        outputs = model(**inputs)
        
        # Calculate loss
        loss = self.bce_loss(outputs['logits'].squeeze(), labels)
        
        return (loss, outputs) if return_outputs else loss


# ============================================================================
# EVALUATION METRICS
# ============================================================================

def compute_metrics(eval_pred):
    """Compute metrics for readmission prediction"""
    predictions, labels = eval_pred
    
    # Extract logits (handle both dict and tensor formats)
    if isinstance(predictions, dict):
        logits = predictions['logits']
    elif isinstance(predictions, tuple):
        logits = predictions[0]
    else:
        logits = predictions
    
    # Convert to numpy if needed
    logits_np = logits.numpy() if isinstance(logits, torch.Tensor) else logits
    
    # Convert logits to probabilities (sigmoid)
    probs = 1 / (1 + np.exp(-logits_np.squeeze()))
    preds = (probs > 0.5).astype(int)
    
    # Handle edge case: if all predictions are same class
    try:
        roc_auc = roc_auc_score(labels, probs)
    except ValueError:
        roc_auc = 0.0
    
    # Calculate metrics
    metrics = {
        'roc_auc': roc_auc,
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, zero_division=0),
        'recall': recall_score(labels, preds, zero_division=0),
        'f1': f1_score(labels, preds, zero_division=0)
    }
    
    return metrics

In [13]:
# ============================================================================
# MAIN TRAINING SCRIPT
# ============================================================================

print("\n" + "=" * 80)
print("FROZEN BACKBONE FINE-TUNING FOR HOSPITAL READMISSION PREDICTION")
print("=" * 80 + "\n")

# Set random seeds
torch.manual_seed(CONFIG.RANDOM_SEED)
np.random.seed(CONFIG.RANDOM_SEED)

# Login to Hugging Face Hub
hf_logged_in = login_to_huggingface(CONFIG)

# Load data
df, pos_weight = load_and_prepare_data(CONFIG)

# Split data
print("\n" + "=" * 80)
print("SPLITTING DATA")
print("=" * 80)
train_df, val_df = train_test_split(
    df, 
    test_size=CONFIG.TEST_SIZE, 
    random_state=CONFIG.RANDOM_SEED,
    stratify=df['readmitted_30day']
)
print(f"✓ Train size: {len(train_df)}")
print(f"✓ Validation size: {len(val_df)}")


FROZEN BACKBONE FINE-TUNING FOR HOSPITAL READMISSION PREDICTION



Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
✓ Successfully logged in to Hugging Face Hub
  Model will be pushed to: AKG2/clinical-longformer-readmission-frozen
LOADING DATA
✓ Loaded admissions: (374139, 16)
✓ Loaded discharge notes: (331731, 7)
✓ Loaded radiology notes: (1144023, 7)

Combining notes...
✓ Final merged data: (374139, 18)
✓ Sampled 100 rows for testing

Preparing labels...
  - Readmission distribution: {0: 79, 1: 21}
  - Positive weight for readmission: 3.76

SPLITTING DATA
✓ Train size: 80
✓ Validation size: 20


In [14]:
# Load tokenizer
print("\n" + "=" * 80)
print("LOADING TOKENIZER")
print("=" * 80)
tokenizer = AutoTokenizer.from_pretrained(CONFIG.MODEL_NAME)
print(f"✓ Loaded tokenizer: {CONFIG.MODEL_NAME}")

# Create datasets
print("\n" + "=" * 80)
print("CREATING DATASETS")
print("=" * 80)
train_dataset = ReadmissionDataset(
    texts=train_df['final_text'].values,
    labels=train_df['readmitted_30day'].values,
    tokenizer=tokenizer,
    max_length=CONFIG.MAX_LENGTH
)

val_dataset = ReadmissionDataset(
    texts=val_df['final_text'].values,
    labels=val_df['readmitted_30day'].values,
    tokenizer=tokenizer,
    max_length=CONFIG.MAX_LENGTH
)
print(f"✓ Train dataset: {len(train_dataset)} samples")
print(f"✓ Validation dataset: {len(val_dataset)} samples")

# Initialize model with frozen backbone
print("\n" + "=" * 80)
print("INITIALIZING MODEL")
print("=" * 80)
model = FrozenReadmissionModel(CONFIG.MODEL_NAME, freeze_backbone=CONFIG.FREEZE_BACKBONE)

# Training arguments
training_args = TrainingArguments(
    output_dir=CONFIG.OUTPUT_DIR,
    num_train_epochs=CONFIG.EPOCHS,
    per_device_train_batch_size=CONFIG.TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=CONFIG.VALID_BATCH_SIZE,
    gradient_accumulation_steps=CONFIG.GRADIENT_ACCUMULATION,
    learning_rate=CONFIG.LEARNING_RATE,
    weight_decay=CONFIG.WEIGHT_DECAY,
    warmup_ratio=CONFIG.WARMUP_RATIO,
    fp16=CONFIG.USE_FP16,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc",
    greater_is_better=True,
    save_total_limit=2,
    report_to="none",
    seed=CONFIG.RANDOM_SEED,
    disable_tqdm=True,
    # Hugging Face Hub settings
    push_to_hub=CONFIG.HF.PUSH_TO_HUB and hf_logged_in,
    hub_model_id=CONFIG.HF.HUB_MODEL_ID if hf_logged_in else None,
    hub_strategy="every_save",
    hub_private_repo=CONFIG.HF.PRIVATE_REPO,
)

# Initialize trainer
print("\n" + "=" * 80)
print("INITIALIZING TRAINER")
print("=" * 80)
trainer = WeightedLossTrainer(
    pos_weight=pos_weight,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)
print("✓ Trainer initialized")

# Train
print("\n" + "=" * 80)
print("TRAINING MODEL (CLASSIFIER ONLY)")
print("=" * 80)
trainer.train()


LOADING TOKENIZER


tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/595M [00:00<?, ?B/s]


🔒 FREEZING BACKBONE WEIGHTS
  - Frozen parameters: 148,659,456
  - Trainable parameters: 769
  - Total parameters: 148,660,225
  - Trainable %: 0.00%

INITIALIZING TRAINER
✓ Trainer initialized

TRAINING MODEL (CLASSIFIER ONLY)


Epoch,Training Loss,Validation Loss,Roc Auc,Accuracy,Precision,Recall,F1
1,No log,1.080237,0.578125,0.200000,0.200000,1.000000,0.333333


TrainOutput(global_step=3, training_loss=7.447104136149089, metrics={'train_runtime': 31.837, 'train_samples_per_second': 2.513, 'train_steps_per_second': 0.094, 'total_flos': 0.0, 'train_loss': 7.447104136149089, 'epoch': 1.0})

In [15]:
# Final evaluation
print("\n" + "=" * 80)
print("FINAL EVALUATION (30-DAY READMISSION)")
print("=" * 80)
eval_results = trainer.evaluate()

print("\n📊 RESULTS:")
print(f"  ROC-AUC:   {eval_results['eval_roc_auc']:.4f}")
print(f"  Accuracy:  {eval_results['eval_accuracy']:.4f}")
print(f"  Precision: {eval_results['eval_precision']:.4f}")
print(f"  Recall:    {eval_results['eval_recall']:.4f}")
print(f"  F1 Score:  {eval_results['eval_f1']:.4f}")

print("\n" + "=" * 80)
print("TRAINING COMPLETE!")
print("=" * 80)


FINAL EVALUATION (30-DAY READMISSION)



📊 RESULTS:
  ROC-AUC:   0.5781
  Accuracy:  0.2000
  Precision: 0.2000
  Recall:    1.0000
  F1 Score:  0.3333

TRAINING COMPLETE!
